In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

print(torch.__version__)

1.10.0+cu111


In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
  root = 'data',
  train = True,
  download = True,
  transform = ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor(),
)

print(training_data)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()


In [4]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data,
                              batch_size = batch_size,)

test_dataloader = DataLoader(test_data,
                             batch_size = batch_size,)

for X, y in test_dataloader:
  print("Shape of X [N, C, H, W]: ", X.shape)
  print("Shape of y: ", y.dtype)
  break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.int64


In [5]:
# creating models
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [7]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28 * 28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)

    return logits

model = NeuralNetwork().to(device)
print(model)  

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
# optimizing the model parameters
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [18]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # backpropagation
    # https://algopoolja.tistory.com/55
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d} / {size:>5d}]")

In [19]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()

  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)

      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accurach: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [20]:
epochs = 5

for t in range(epochs):
  print(f"Epoch {t+1}\n----------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)

print("Done")

Epoch 1
----------------
loss: 1.950661 [    0 / 60000]
loss: 1.924185 [ 6400 / 60000]
loss: 1.823136 [12800 / 60000]
loss: 1.865593 [19200 / 60000]
loss: 1.758718 [25600 / 60000]
loss: 1.694482 [32000 / 60000]
loss: 1.723889 [38400 / 60000]
loss: 1.619089 [44800 / 60000]
loss: 1.642708 [51200 / 60000]
loss: 1.541492 [57600 / 60000]
Test Error: 
 Accurach: 60.1%, Avg loss: 1.554870 

Epoch 2
----------------
loss: 1.614534 [    0 / 60000]
loss: 1.573278 [ 6400 / 60000]
loss: 1.434398 [12800 / 60000]
loss: 1.504510 [19200 / 60000]
loss: 1.382329 [25600 / 60000]
loss: 1.367559 [32000 / 60000]
loss: 1.387772 [38400 / 60000]
loss: 1.306288 [44800 / 60000]
loss: 1.344147 [51200 / 60000]
loss: 1.243647 [57600 / 60000]
Test Error: 
 Accurach: 62.4%, Avg loss: 1.267864 

Epoch 3
----------------
loss: 1.342106 [    0 / 60000]
loss: 1.315557 [ 6400 / 60000]
loss: 1.161108 [12800 / 60000]
loss: 1.265915 [19200 / 60000]
loss: 1.138649 [25600 / 60000]
loss: 1.158226 [32000 / 60000]
loss: 1.185285 

In [21]:
# saving models
torch.save(model.state_dict(), 'model.pth')
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [22]:
# Loading models
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [24]:
# This model can now be used to make predictions.
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()

x, y = test_data[0][0], test_data[0][1]

with torch.no_grad():
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'predicted: "{predicted}", Actual: "{actual}"')

predicted: "Ankle boot", Actual: "Ankle boot"
